In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import datetime

In [2]:
# Cargar el dataset y preprocesar
df = pd.read_csv('../../../data/processed/train_processed.csv')
df

,NewExist,UrbanRural,RevLineCr,LowDoc,Accept,BankStateInOhio,ApprovalDateMonth,ApprovalFYGrouped,NoEmpGrouped,CreateJobBinary,RetainedJobBinary,IsFranchise,DisbursementGrossGrouped
0,2,1,0,0,1,0,11,2006,0,0,1,0,0
1,2,1,0,0,1,1,6,2005,0,1,1,0,1
2,2,2,1,0,1,1,3,2003,0,1,1,0,0
3,2,0,0,0,1,1,6,1995,0,0,0,0,1
4,1,1,0,0,0,1,4,2009,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14429,1,0,0,0,1,0,11,1994,1,0,0,0,2
14430,2,1,1,0,1,0,9,2008,0,1,1,0,1
14431,1,1,0,0,1,0,12,2000,1,0,0,0,1
14432,2,1,0,0,1,1,4,2007,0,1,0,1,2


# Preparación de los datos 

Bajo las mismas premisas que en el notebook [1_CatBoost_Balanceado](notebooks/3_modelado/3.2_modelos_de_arboles/1_CatBoost_Balanceado.ipynb), y tras los resultados obtenidos en la observación de importancia de variables, se imputa la variable `JobCombo`:

In [3]:
# Creamos la nueva variable, eliminamos las anteriores y observamos la distribucción de la variable objetivo
df["JobCombo"] = df["CreateJobBinary"].astype(str) + "_" + df["RetainedJobBinary"].astype(str)
pd.crosstab(df["JobCombo"], df["Accept"], normalize='index')

Accept,0,1
JobCombo,,
0_0,0.174546,0.825454
0_1,0.251936,0.748064
1_0,0.100073,0.899927
1_1,0.233402,0.766598


# Parametrización del modelo inicial

Se definen las variables categóricas del modelo

In [4]:
# Selección de las variables categóricas a evaluar
cat_features = [
    #'City', 
    #'CityBankGrouped',
    #'CityGrouped',
    #'BankState', 
    'BankStateInOhio',
    'UrbanRural',
    #'NoEmp', 
    #'NoEmpGrouped',
    #'CreateJob',
    #'CreateJobBinary',
    #'RetainedJob',
    #'RetainedJobBinary',
    #'NewExist', 
    #'RevLineCr',
    #'LowDoc',
    #'Accept',
    #'ApprovalDateMonth',
    'ApprovalFYGrouped',
    #'IsFranchise',
    'DisbursementGrossGrouped',
    'JobCombo',
    #'ApprovalFYBin'
]
#drop_columns = ["Accept"]
drop_columns = ["Accept","CreateJobBinary","RetainedJobBinary","IsFranchise","NoEmpGrouped","ApprovalDateMonth","LowDoc","NewExist","RevLineCr"]

## División en conjunto de test y entrenamiento

In [5]:
X = df.copy().drop(columns=drop_columns)
y = df["Accept"]

# División en conjunto de test y entrenamiento
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from collections import Counter

from collections import Counter

# 1. Contar clases
class_counts = Counter(y_train)
print("Distribución de clases:", class_counts)

# 2. Calcular pesos manuales
total_samples = len(y_train)
n_classes = len(class_counts)

weights = {
    cls: total_samples / (n_classes * count)
    for cls, count in class_counts.items()
}

print("Pesos calculados:", weights)


Distribución de clases: Counter({1: 9185, 0: 2362})
Pesos calculados: {0: 2.4443268416596107, 1: 0.6285792052259118}


In [27]:
# ------------------------------------------
# Parámetros del modelo - Ajustados con GridSearchCV y Random
# ------------------------------------------
params = {
    'iterations': 500,
    'learning_rate': 0.05,
    'depth': 6,
    'l2_leaf_reg': 4,
    #'loss_function': 'Logloss',
    'eval_metric': 'F1',
    #'auto_class_weights':'Balanced',
    'class_weights':[2.5,1],
    'random_state': 42,
    'verbose': 100
}

# Crear el modelo usando **params
model = CatBoostClassifier(**params)
model.fit(X_train, y_train, cat_features=cat_features)

0:	learn: 0.7210031	total: 11.9ms	remaining: 5.95s
100:	learn: 0.7632614	total: 629ms	remaining: 2.49s
200:	learn: 0.7662354	total: 1.22s	remaining: 1.81s
300:	learn: 0.7740664	total: 1.72s	remaining: 1.14s
400:	learn: 0.7778593	total: 2.28s	remaining: 563ms
499:	learn: 0.7828489	total: 2.78s	remaining: 0us


## Evaluacion del modelo

In [28]:
# Evaluación
y_pred = model.predict(X_test)
print("F1_score:", f1_score(y_test, y_pred))
print("\nClasificación:\n", classification_report(y_test, y_pred))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))

F1_score: 0.8425438596491227

Clasificación:
               precision    recall  f1-score   support

           0       0.38      0.44      0.41       564
           1       0.86      0.83      0.84      2323

    accuracy                           0.75      2887
   macro avg       0.62      0.63      0.63      2887
weighted avg       0.77      0.75      0.76      2887


Matriz de confusión:
 [[ 248  316]
 [ 402 1921]]


El modelo predice en un porcentaje muy bajo los casos positivos de la clase 0. Se trata de mejorar con el umbral e ingeniería de categorías pero se obtienen resultados similares.

In [29]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# 1. Definición del modelo base
model = CatBoostClassifier(verbose=100, class_weights=[2.5,1],random_state=42, eval_metric="F1")

# 2. Definí el grid de hiperparámetros a probar
param_grid = {
    'iterations':[200,300,500],
    'depth':[4,6],
    #'learning_rate':[0.03,0.05],
    'l2_leaf_reg': [4,6,8],
}

# 3. Usar K-fold estratificado para clases desbalanceadas
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 4. Búsqueda de hiperparámetros
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=cv,
    n_jobs=-1,
    verbose=True
)

# 5. Ajuste
grid.fit(X_train, y_train, cat_features=cat_features)

# 6. Ver los mejores hiperparámetros
print("Mejores hiperparámetros:")
print(grid.best_params_)
print("Mejor puntuación:", grid.best_score_)


Fitting 3 folds for each of 18 candidates, totalling 54 fits
0:	learn: 0.7210031	total: 4.89ms	remaining: 2.44s
100:	learn: 0.7617753	total: 466ms	remaining: 1.84s
200:	learn: 0.7601707	total: 1s	remaining: 1.49s
300:	learn: 0.7594659	total: 1.43s	remaining: 949ms
400:	learn: 0.7613912	total: 1.89s	remaining: 467ms
499:	learn: 0.7622672	total: 2.37s	remaining: 0us
Mejores hiperparámetros:
{'depth': 4, 'iterations': 500, 'l2_leaf_reg': 8}
Mejor puntuación: 0.6281703000969834


In [30]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import numpy as np
best_model = grid.best_estimator_
# 1. Obtener probabilidades para clase 1
y_test_proba = best_model.predict_proba(X_test)[:, 1]

# 2. Buscar el umbral que maximiza el F1-score
best_threshold = 0.5
best_f1 = 0
f1_scores = []
thresholds = np.linspace(0.1, 0.9, 100)

for t in thresholds:
    y_pred = (y_test_proba >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    f1_scores.append(f1)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

# 3. Mostrar resultado
print(f"Mejor F1-score: {best_f1:.4f} con umbral = {best_threshold:.2f}")


Mejor F1-score: 0.8919 con umbral = 0.10


No se hace uso del umbral porque baja el F1 score en la estimación

In [31]:
df_test_id = pd.read_csv("../../../data/processed/test_nolabel_processed.csv")

#drop_columns.append("id")
#drop_columns.remove("Accept")

# Se añaden y eliminan las columnas correspondientes para que ambos datasets tengan la misma estructura
df_test_id["JobCombo"] = df_test_id["CreateJobBinary"].astype(str) + "_" + df_test_id["RetainedJobBinary"].astype(str)
df_test = df_test_id.copy().drop(columns=drop_columns)

test_pool = Pool(df_test, cat_features=cat_features)

df_test_id["Accept"] = best_model.predict(test_pool)

# Crea el archivo CSV con las columnas requeridas: 'id' y 'Accept'
filename = f"CatBoost_SIMPLE_{datetime.datetime.now().strftime('%Y%m%d_%H_%M_%S')}.csv"
df_test_id.to_csv(filename, columns=['id', 'Accept'], index=False)

print("Archivo de submission ", filename, " generado correctamente.")


Archivo de submission  CatBoost_SIMPLE_20250410_13_28_10.csv  generado correctamente.
0:	learn: 0.7442068	total: 54.9ms	remaining: 10.9s
100:	learn: 0.7598144	total: 1.33s	remaining: 1.3s
199:	learn: 0.7600271	total: 2.9s	remaining: 0us
0:	learn: 0.7381078	total: 7.1ms	remaining: 1.41s
100:	learn: 0.7620929	total: 923ms	remaining: 905ms
199:	learn: 0.7615850	total: 1.63s	remaining: 0us
0:	learn: 0.7381078	total: 6.53ms	remaining: 1.3s
100:	learn: 0.7599266	total: 748ms	remaining: 733ms
199:	learn: 0.7587588	total: 2.27s	remaining: 0us
0:	learn: 0.7442068	total: 4.18ms	remaining: 1.25s
100:	learn: 0.7605084	total: 924ms	remaining: 1.82s
200:	learn: 0.7603131	total: 2.09s	remaining: 1.03s
299:	learn: 0.7624957	total: 3.33s	remaining: 0us
0:	learn: 0.7448235	total: 5.32ms	remaining: 1.59s
100:	learn: 0.7625529	total: 1.03s	remaining: 2.04s
200:	learn: 0.7608823	total: 2.03s	remaining: 998ms
299:	learn: 0.7607344	total: 3.16s	remaining: 0us
0:	learn: 0.7448235	total: 12.5ms	remaining: 6.23

# Conclusiones:
- En este Notebook se han probado dos tipos de balanceo, sobre el dataset sin balancear en el procesado, obteniendo mejores resultados mediante el ajuste automático del modelo (`auto_class_weights`=`Balanced`) que con el ajuste manual estimando los valores del parámetro `class_weights` en base al desbalance que presenta la clase 0 frente a la clase 1
- El ajuste del umbral de decisión no ha dado buenos resultados
- El mejor resultado encontrado ha sido mediante búsqueda de hiperparámetros, sin ajuste de umbral y con ajuste de pesos automático: 67,68%
